<a href="https://colab.research.google.com/github/zooodung/TA-Prediction/blob/main/Preprocess_BeforeEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교통사고 위험도 예측

## 라이브러리 및 파일 호출

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytimekr import pytimekr
from sklearn.preprocessing import LabelEncoder
import os

In [3]:
path=os.getcwd()

In [4]:
daegu_data=pd.read_csv(path + f'/데이터들/전처리 전/train.csv')
wide_data=pd.read_csv(path + f'/데이터들/전처리 전/external_open/countrywide_accident.zip')
test_data=pd.read_csv(path + f'/데이터들/전처리 전/test.csv')

## 기초 통계 분석

In [5]:
#ta_data_daegu.info()

In [6]:
#x_test.info()

In [7]:
#ta_data_daegu.describe()

In [8]:
#ta_data_daegu.describe()

In [9]:
#columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형']

#for column in columns:
#    print(f"Column: {column}")
#    print(ta_data_daegu[column].value_counts())
#    print("-" * 30)

## 전처리

### 대구 데이터 전처리

#### 불필요한 컬럼 제거

In [10]:
columns_to_drop = ['ID', '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령',
                   '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']


daegu=daegu_data.drop(columns=columns_to_drop)

#### 사고일시 컬럼 연도, 월, 일, 시간 단위로 분할

In [11]:
#daegu[['년도','월','일+시간']] = daegu['사고일시'].str.split('-',expand=True)
#daegu[['일','시간']]= daegu['일+시간'].str.split(' ',expand=True)

#daegu.drop(['일+시간','사고일시'],inplace=True, axis=1)
#daegu.drop(['일'],inplace=True, axis=1)

#### 시군구 단위로 분할

In [12]:
daegu[['시','구','동']] = daegu['시군구'].str.split(' ',expand=True)
daegu.drop(['시군구'],inplace=True, axis=1)

#### 도로 형태 분할

In [13]:
daegu[['도로형태1', '도로형태2']] = daegu['도로형태'].str.split(' - ', expand=True)
daegu.drop(['도로형태'],inplace=True, axis=1)

In [14]:
daegu.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타


#### 요일 컬럼 평일과 주말로 분류

In [15]:
#ta_data_daegu['요일 분류'] = ta_data_daegu['요일'].apply(lambda x: '평일' if x in ['월요일', '화요일', '수요일', '목요일', '금요일'] else '주말')

In [16]:
#ta_data_daegu

### 전국 데이터 전처리

In [17]:
wide=wide_data.copy()
len(wide)

602775

#### 철길 건널목 제거

In [18]:
wide = wide[~wide['도로형태'].str.split().str[-1].str.contains('철길건널목')]

#### 불필요한 컬럼 제거

In [19]:
wide=wide.drop(columns=columns_to_drop)

#### 광역시, 특별시만 분류

In [20]:
def contains_specific_location(address):
    cities = ['서울특별시', '인천광역시', '부산광역시', '울산광역시', '광주광역시', '대전광역시']
    for city in cities:
        if city in address:
            return True
    return False
wide=wide[wide['시군구'].apply(contains_specific_location)]

wide[['시','구','동']] = wide['시군구'].str.split(' ',expand=True)

In [21]:
wide['시군구'].apply(lambda x:x.split()[0]).value_counts()

시군구
서울특별시    108395
부산광역시     36244
인천광역시     25115
광주광역시     23430
대전광역시     22579
울산광역시     11981
Name: count, dtype: int64

In [22]:
wide.drop(['시군구'],inplace=True, axis=1)

#### 연도, 월, 일, 시간 분류

In [23]:
#wide[['년도','월','일+시간']] = wide['사고일시'].str.split('-',expand=True)
#wide[['일','시간']]= wide['일+시간'].str.split(' ',expand=True)

#wide.drop(['일+시간','사고일시'],inplace=True, axis=1)
#wide.drop(['일'],inplace=True, axis=1)

#### 도로 형태 분류

In [24]:
wide[['도로형태1', '도로형태2']] = wide['도로형태'].str.split(' - ', expand=True)

wide.drop(['도로형태'],inplace=True, axis=1)

In [25]:
wide.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,강서구,방화동,교차로,교차로횡단보도내
4,2019-01-01 00,화요일,맑음,건조,차대차,0,0,2,1,7,인천광역시,부평구,부평동,교차로,교차로안
5,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,인천광역시,부평구,부평동,교차로,교차로부근
8,2019-01-01 00,화요일,맑음,건조,차대차,0,0,1,0,3,광주광역시,광산구,수완동,단일로,기타
11,2019-01-01 00,화요일,맑음,건조,차대사람,1,0,0,0,10,부산광역시,해운대구,우동,교차로,교차로안


### 데이터 병합 후 전처리 진행

In [26]:
print(len(daegu),len(wide), len(daegu)+len(wide))

39609 227744 267353


In [27]:
concatenated_df = pd.concat([daegu, wide])
concatenated_df

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,2021-12-31 23,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타
602769,2021-12-31 23,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타
602770,2021-12-31 23,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안
602771,2021-12-31 23,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타


#### 휴일 여부

In [28]:
for i in range(2019, 2024):
    kr_holidays = pytimekr.holidays(i)
    print(kr_holidays)

[datetime.date(2019, 2, 4), datetime.date(2019, 2, 5), datetime.date(2019, 2, 6), datetime.date(2019, 9, 12), datetime.date(2019, 9, 13), datetime.date(2019, 9, 14), datetime.date(2019, 1, 1), datetime.date(2019, 3, 1), datetime.date(2019, 5, 5), datetime.date(2019, 5, 12), datetime.date(2019, 6, 6), datetime.date(2019, 8, 15), datetime.date(2019, 10, 9), datetime.date(2019, 10, 3), datetime.date(2019, 12, 25)]
[datetime.date(2020, 1, 24), datetime.date(2020, 1, 25), datetime.date(2020, 1, 26), datetime.date(2020, 9, 30), datetime.date(2020, 10, 1), datetime.date(2020, 10, 2), datetime.date(2020, 1, 1), datetime.date(2020, 3, 1), datetime.date(2020, 5, 5), datetime.date(2020, 4, 30), datetime.date(2020, 6, 6), datetime.date(2020, 8, 15), datetime.date(2020, 10, 9), datetime.date(2020, 10, 3), datetime.date(2020, 12, 25)]
[datetime.date(2021, 2, 11), datetime.date(2021, 2, 12), datetime.date(2021, 2, 13), datetime.date(2021, 9, 20), datetime.date(2021, 9, 21), datetime.date(2021, 9, 22)

In [29]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [30]:
concatenated_df['사고일시'] = pd.to_datetime(concatenated_df['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
concatenated_df['년도'] = concatenated_df['사고일시'].dt.year
concatenated_df['월'] = concatenated_df['사고일시'].dt.month
concatenated_df['day'] = concatenated_df['사고일시'].dt.day
concatenated_df['시간'] = concatenated_df['사고일시'].dt.hour
concatenated_df['day_of_week'] = concatenated_df['사고일시'].dt.dayofweek

In [31]:
concatenated_df['holiday'] = np.where((concatenated_df.day_of_week >= 5) | (concatenated_df.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

In [32]:
concatenated_df.drop(columns=['사고일시'], inplace=True)

In [33]:
concatenated_df['holiday'].value_counts()

holiday
평일    195393
휴일     71960
Name: count, dtype: int64

In [34]:
concatenated_df.drop(['day', 'day_of_week'], inplace=True, axis=1)

#### 주말 여부

In [35]:
#concatenated_df['주말여부']=concatenated_df['요일'].apply(lambda x:'주말' if x in ['토요일','일요일'] else '평일')

#concatenated_df['주말여부'].value_counts()

#### 시간 분류

In [36]:
def seperate_time(time): #심야, 새벽, 오전, 오후, 저녁, 야간 ->4시간단위
    if 0<=int(time)<4:
        return '심야'
    elif int(time)<8:
        return '새벽'
    elif int(time)<12:
        return '오전'
    elif int(time)<16:
        return '오후'
    elif int(time)<20:
        return '저녁'
    else:
        return '야간'
concatenated_df['시간구분']=concatenated_df['시간'].apply(seperate_time)
print(concatenated_df['시간구분'].value_counts().sum())
concatenated_df['시간구분'].value_counts()

267353


시간구분
저녁    71768
오후    58743
오전    50058
야간    47737
새벽    20554
심야    18493
Name: count, dtype: int64

#### 계절 분류

In [37]:
def seperate_month(month):
    if int(month) in [12,1,2]:
        return '겨울'
    elif int(month) in [3,4,5]:
        return '봄'
    elif int(month) in [6,7,8]:
        return '여름'
    else:
        return '가을'
concatenated_df['계절']=concatenated_df['월'].apply(seperate_month)
print(concatenated_df['계절'].value_counts().sum())
concatenated_df['계절'].value_counts()

267353


계절
가을    70234
여름    68434
봄     66607
겨울    62078
Name: count, dtype: int64

### 병합된 데이터 전처리 확인

In [38]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타,2021,12,23,평일,야간,겨울


### 테스트 데이터 전처리 진행

In [39]:
test=test_data.copy()

In [40]:
test[['시','구','동']] = test['시군구'].str.split(' ',expand=True)
test.drop(['시군구'],inplace=True, axis=1)

In [41]:
test[['도로형태1', '도로형태2']] = test['도로형태'].str.split(' - ', expand=True)
test.drop(['도로형태'],inplace=True, axis=1)

In [42]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [43]:
test['사고일시'] = pd.to_datetime(test['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
test['년도'] = test['사고일시'].dt.year
test['월'] = test['사고일시'].dt.month
test['day'] = test['사고일시'].dt.day
test['시간'] = test['사고일시'].dt.hour
test['day_of_week'] = test['사고일시'].dt.dayofweek

test['holiday'] = np.where((test.day_of_week >= 5) | (test.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

test.drop(columns=['사고일시'], inplace=True)
test.drop(['day', 'day_of_week'], inplace=True, axis=1)

In [44]:
test['시간구분']=test['시간'].apply(seperate_time)

In [45]:
test['계절']=test['월'].apply(seperate_month)
test['계절'].value_counts()

계절
가을    2947
여름    2783
봄     2771
겨울    2462
Name: count, dtype: int64

#### 테스트 데이터 전처리 결과 확인

In [46]:
test

,ID,요일,기상상태,노면상태,사고유형,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,대구광역시,수성구,상동,교차로,교차로안,2022,1,1,휴일,심야,겨울
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,대구광역시,수성구,지산동,단일로,기타,2022,1,1,휴일,심야,겨울
2,ACCIDENT_39611,토요일,맑음,건조,차대차,대구광역시,수성구,수성동2가,교차로,교차로안,2022,1,4,휴일,새벽,겨울
3,ACCIDENT_39612,토요일,맑음,건조,차대차,대구광역시,수성구,신매동,단일로,기타,2022,1,4,휴일,새벽,겨울
4,ACCIDENT_39613,토요일,맑음,건조,차대차,대구광역시,달서구,감삼동,교차로,교차로안,2022,1,6,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,토요일,맑음,건조,차대차,대구광역시,남구,대명동,단일로,터널,2022,12,18,휴일,저녁,겨울
10959,ACCIDENT_50568,토요일,맑음,건조,차대차,대구광역시,수성구,시지동,단일로,기타,2022,12,18,휴일,저녁,겨울
10960,ACCIDENT_50569,토요일,맑음,건조,차대차,대구광역시,수성구,연호동,단일로,기타,2022,12,20,휴일,야간,겨울
10961,ACCIDENT_50570,토요일,맑음,건조,차대차,대구광역시,수성구,범물동,교차로,교차로부근,2022,12,20,휴일,야간,겨울


#### 합쳐진 훈련 데이터 셋 오류 수정

In [50]:
concatenated_df = concatenated_df[~concatenated_df['기상상태'].str.split().str[-1].str.contains('안개')]
concatenated_df = concatenated_df[~concatenated_df['노면상태'].str.split().str[-1].str.contains('해빙')]

In [51]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타,2021,12,23,평일,야간,겨울


#### 전처리 데이터 저장 - 전처리 매번 로딩할 필요 없다

In [ ]:
concatenated_df.to_csv(path + f'/데이터들/인코딩 없음/train.csv')
test.to_csv(path + f'/데이터들/인코딩 없음/test.csv')

In [ ]:
path = os.getcwd()

concatenated_df.to_csv(path + f'/데이터들/인코딩 없음/train.csv')
test = pd.read_csv(path + f'/데이터들/인코딩 없음/test.csv')